In [1]:
%matplotlib inline
import os
import re
import io
import itertools

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *


import IPython.display as display

In [2]:
output_notebook()

Loading BokehJS ...

In [3]:
%run param.py

In [4]:
# clirad's flux and cooling rate
PATH_FLUX_CLIRAD = os.path.join(DIR_FORTRAN, 'output_flux.dat')
PATH_COOLR_CLIRAD = os.path.join(DIR_FORTRAN, 'output_coolr.dat')

In [5]:
# lblnew's CRD flux and cooling rate
PATH_FLUX_LBLNEW = os.path.join(DIR_FORTRAN_LBLNEW, 'output_flux.dat')
PATH_COOLR_LBLNEW = os.path.join(DIR_FORTRAN_LBLNEW, 'output_coolr.dat')

In [6]:
# lblnew's k-dist, transf/diffusivity flux and cooling rate
PATH_WFLUX_LBLNEW = os.path.join(DIR_FORTRAN_LBLNEW, 'output_wflux.dat')
PATH_WCOOLR_LBLNEW = os.path.join(DIR_FORTRAN_LBLNEW, 'output_wcoolr.dat')

In [7]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)
    return ds

In [8]:
DS_FLUX_CLIRAD = load_output_file(PATH_FLUX_CLIRAD)
DS_COOLR_CLIRAD = load_output_file(PATH_COOLR_CLIRAD)

In [9]:
DS_FLUX_LBLNEW = load_output_file(PATH_FLUX_LBLNEW)
DS_COOLR_LBLNEW = load_output_file(PATH_COOLR_LBLNEW)
DS_WFLUX_LBLNEW = load_output_file(PATH_WFLUX_LBLNEW)
DS_WCOOLR_LBLNEW = load_output_file(PATH_WCOOLR_LBLNEW)

In [10]:
def pltdata_cooling_1(ds_coolr_clirad=None,
                      ds_coolrg_lblnew=None, ds_wcoolrg_lblnew=None,
                      ds_coolr_lblnew=None, ds_wcoolr_lblnew=None):
    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': ds_coolr_clirad['coolrg'].sel(band=11),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': ds_coolr_lblnew['coolrg'],
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': ds_wcoolr_lblnew['coolrg'].sel(igg=10),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data



def nice_xlims(pltdata=None, prange=None):
    
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)
    

def plt_cooling_bokeh(pltdata=None):
    '''
    Plot a list of cooling rate profiles using bokeh.
    '''
    ymin = 1e-2 
    ymax = 1020 
        
    p1 = figure(title="Linear pressure scale", 
                plot_width=400)        
    xmin, xmax = nice_xlims(pltdata, prange=(50, 1050))
    for d in pltdata:
        if 'marker' in d:
            getattr(p1, d['marker'])(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=.7)
        p1.line(d['srs'].values, 
                d['srs'].coords['pressure'].values,
                color=d['color'], alpha=d['alpha'], 
                line_width=d['line_width'], line_dash=d['line_dash'])
    
    p1.y_range = Range1d(ymax, ymin)
    p1.yaxis.axis_label = 'pressure [mb]'   
    p1.x_range = Range1d(xmin, xmax)
    p1.xaxis.axis_label = 'cooling rate [K/day]'
            
    p2 = figure(title='Log pressure scale', y_axis_type='log',
                plot_width=560)
    xmin, xmax = nice_xlims(pltdata, prange=(.01, 200))
    
    rs = []
    for d in pltdata:
        rd = []
        if 'marker' in d:
            r_mark = getattr(p2, d['marker'])(d['srs'].values, 
                        d['srs'].coords['pressure'].values,
                        color=d['color'], alpha=.7)
            rd.append(r_mark)
        r_line = p2.line(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=d['alpha'], 
                    line_width=d['line_width'], line_dash=d['line_dash'])
        rd.append(r_line)
      
        rs.append(rd)
        
    p2.y_range = Range1d(ymax, ymin)  
    p2.yaxis.axis_label = 'pressure [mb]'
    
    p2.x_range = Range1d(xmin, xmax)
    p2.xaxis.axis_label = 'cooling rate [K/day]'
    
    items = [(d['label'], r) for r, d in zip(rs, pltdata)]
    legend = Legend(items=items, location=(10, -30))
    legend.label_text_font_size = '8pt'
    
    p2.add_layout(legend, 'right')  
    
    show(gridplot(p1, p2, ncols=2, plot_height=500))
    

def script_plt_cooling():
    pltdata = pltdata_cooling_1(
        ds_coolr_clirad=DS_COOLR_CLIRAD,
        ds_coolr_lblnew=DS_COOLR_LBLNEW,
        ds_wcoolr_lblnew=DS_WCOOLR_LBLNEW)
    plt_cooling_bokeh(pltdata=pltdata)
    print('''FIGURE. Cooling rate profiles.''')

In [11]:

def tbdata_flux(ds_flux_clirad=None,
                ds_fluxg_lblnew=None, ds_wfluxg_lblnew=None,
                ds_flux_lblnew=None, ds_wflux_lblnew=None):
    ilevels = [0, 23, -1]
    
    def fmt(ds):
        return ds.isel(pressure=ilevels)\
                 .to_dataframe()\
                 .set_index('level', append=True)\
                 .drop('igg', axis=1, errors='ignore')\
                 .drop('band', axis=1, errors='ignore')\
                 .groupby(['pressure', 'level']).sum()
            
    benchmark = {'label': 'WGT igg=10 wgt_flux=1',
                 'df': fmt(ds_wflux_lblnew.sel(igg=10))}
    
    
    tbs = [{'label': 'CRD',
            'df': fmt(ds_flux_lblnew)},
           {'label': 'CLIRAD',
            'df': fmt(ds_flux_clirad.sel(band=11))}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data


def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        print('Table.', 'Fluxes.', d['label'])
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        print('Table.', 'Fluxes.', 
              '({}) - ({})'.format(d['label'], d0['label']))
        
    show_tb(benchmark)
    for d in others:
        show_tb(d)
        
    for d in others:
        show_tbdiff(d=d, d0=benchmark)
        
        
def script_tb_flux():
    data = tbdata_flux(ds_flux_clirad=DS_FLUX_CLIRAD,
                       ds_flux_lblnew=DS_FLUX_LBLNEW,
                       ds_wflux_lblnew=DS_WFLUX_LBLNEW)
    tb_flux(tbdata=data)

In [12]:
def script():
    script_plt_cooling()
    script_tb_flux()

In [13]:
script()

FIGURE. Cooling rate profiles.


,,flug,fldg,fnetg
pressure,level,,,
0.0000,1,-4.023297,0.000000,-4.023297
1.0685,24,-4.030783,0.017237,-4.013546
1013.0000,76,-6.625452,4.062985,-2.562467


Table. Fluxes. WGT igg=10 wgt_flux=1


,,flug,fldg,fnetg
pressure,level,,,
0.0000,1,-4.087499,0.000000,-4.087499
1.0685,24,-4.094944,0.018859,-4.076085
1013.0000,76,-6.625452,4.061761,-2.563691


Table. Fluxes. CRD


...

Table. Fluxes. CLIRAD


,,flug,fldg,fnetg
pressure,level,,,
0.0000,1,-0.064203,0.000000,-0.064203
1.0685,24,-0.064161,0.001622,-0.062539
1013.0000,76,0.000000,-0.001224,-0.001224


Table. Fluxes. (CRD) - (WGT igg=10 wgt_flux=1)


,,flug,fldg,fnetg
pressure,level,,,
0.0000,1,6.976000e-04,0.000002,0.000699
1.0685,24,5.990000e-04,0.000405,0.001005
1013.0000,76,3.000000e-07,0.005991,0.005992


Table. Fluxes. (CLIRAD) - (WGT igg=10 wgt_flux=1)


In [14]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')